<a href="https://colab.research.google.com/github/AnalyticsKnight/yemoonsaBigdata/blob/dev/P2_C4_04_CensusIncome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/202201_bigdata_share

/content/drive/MyDrive/202201_bigdata_share


# **Chapter 4. A to Z 빅데이터 분석 실습**

## 3. 실습 A to Z - Census Income 

### (2) 사전준비

#### 432-01 : 패키지 불러오기 

In [ ]:
import pandas as pd
import numpy as np

432-02 : 데이터 로딩

In [ ]:
X_test = pd.read_csv("dataset/census_X_test.csv")
X_train = pd.read_csv("dataset/census_X_train.csv")
y_train = pd.read_csv("dataset/census_y_train.csv")

### (3) 데이터 전처리

#### 433-01 : 데이터 정보 확인 

In [ ]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   education_num   30162 non-null  int64 
 3   marital_status  30162 non-null  object
 4   occupation      30162 non-null  object
 5   relationship    30162 non-null  object
 6   race            30162 non-null  object
 7   sex             30162 non-null  object
 8   capital_gain    30162 non-null  int64 
 9   capital_loss    30162 non-null  int64 
 10  hours_per_week  30162 non-null  int64 
 11  native_country  30162 non-null  object
dtypes: int64(5), object(7)
memory usage: 2.8+ MB
None


433-02 : 데이터 통계량 확인

In [ ]:
print(X_train.describe())

                age  education_num  capital_gain  capital_loss  hours_per_week
count  30162.000000   30162.000000  30162.000000  30162.000000    30162.000000
mean      38.437902      10.121312   1092.007858     88.372489       40.931238
std       13.134665       2.549995   7406.346497    404.298370       11.979984
min       17.000000       1.000000      0.000000      0.000000        1.000000
25%       28.000000       9.000000      0.000000      0.000000       40.000000
50%       37.000000      10.000000      0.000000      0.000000       40.000000
75%       47.000000      13.000000      0.000000      0.000000       45.000000
max       90.000000      16.000000  99999.000000   4356.000000       99.000000


#### 433-03 : capital_gain변수 통계량 확인

In [ ]:
print(X_train['capital_gain'].quantile([q/20 for q in range(15, 21)]))

0.75        0.0
0.80        0.0
0.85        0.0
0.90        0.0
0.95     5013.0
1.00    99999.0
Name: capital_gain, dtype: float64


#### 433-04 : capital_loss변수 통계량 확인

In [ ]:
print(X_train['capital_loss'].quantile([q/20 for q in range(15, 21)]))

0.75       0.0
0.80       0.0
0.85       0.0
0.90       0.0
0.95       0.0
1.00    4356.0
Name: capital_loss, dtype: float64


#### 433-05 : 파생변수 생성

In [ ]:
X_train['capital_gain_yn'] = np.where(X_train['capital_gain']>0, 1, 0)
X_train['capital_loss_yn'] = np.where(X_train['capital_loss']>0, 1, 0)

X_test['capital_gain_yn'] = np.where(X_test['capital_gain']>0, 1, 0)
X_test['capital_loss_yn'] = np.where(X_test['capital_loss']>0, 1, 0)

#### 433-06 : 변수 구분 

In [ ]:
# 불필요 변수
COL_DEL = []
# 수치형 변수
COL_NUM = ['age', 'education_num', 'hours_per_week', 'capital_gain', 'capital_loss']
# 범주형 변수 
COL_CAT = ['workclass', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'capital_gain_yn', 'capital_loss_yn']
# 종속 변수
COL_Y = ['target']

X_train = X_train.drop(COL_DEL, axis=1)
X_test = X_test.drop(COL_DEL, axis=1)

#### 433-07 : 수치형 변수 탐색

In [ ]:
train_df = pd.concat([X_train, y_train], axis=1)

for _col in COL_NUM:
  print('-'*80)
  print(_col)
  print(train_df.groupby(COL_Y)[_col].describe(), end='\n\n')

--------------------------------------------------------------------------------
age
          count      mean        std   min   25%   50%   75%   max
target                                                            
0       22654.0  36.60806  13.464631  17.0  26.0  34.0  45.0  90.0
1        7508.0  43.95911  10.269633  19.0  36.0  43.0  51.0  90.0

--------------------------------------------------------------------------------
education_num
          count       mean       std  min   25%   50%   75%   max
target                                                           
0       22654.0   9.629116  2.413596  1.0   9.0   9.0  10.0  16.0
1        7508.0  11.606420  2.368423  2.0  10.0  12.0  13.0  16.0

--------------------------------------------------------------------------------
hours_per_week
          count       mean        std  min   25%   50%   75%   max
target                                                            
0       22654.0  39.348592  11.950774  1.0  38.0  40.0  

#### 433-08 : 범주형 변수 탐색

In [ ]:
for _col in COL_CAT:
  print(train_df.groupby(_col, as_index=False)[COL_Y].mean().sort_values(by=COL_Y, ascending=False), end='\n\n')

          workclass    target
3      Self-emp-inc  0.558659
0       Federal-gov  0.387063
1         Local-gov  0.294630
4  Self-emp-not-inc  0.285714
5         State-gov  0.268960
2           Private  0.218792
6       Without-pay  0.000000

          marital_status    target
1      Married-AF-spouse  0.476190
2     Married-civ-spouse  0.454959
0               Divorced  0.107262
6                Widowed  0.096735
3  Married-spouse-absent  0.083784
5              Separated  0.070288
4          Never-married  0.048324

           occupation    target
3     Exec-managerial  0.485220
9      Prof-specialty  0.448489
10    Protective-serv  0.326087
12       Tech-support  0.304825
11              Sales  0.270647
2        Craft-repair  0.225310
13   Transport-moving  0.202926
0        Adm-clerical  0.133835
6   Machine-op-inspct  0.124619
4     Farming-fishing  0.116279
1        Armed-Forces  0.111111
5   Handlers-cleaners  0.061481
7       Other-service  0.041096
8     Priv-house-serv  0.00699

#### 433-09 : 범주형 변수 - 레이블 인코딩

In [ ]:
from sklearn.preprocessing import LabelEncoder

for _col in COL_CAT:
  le = LabelEncoder()
  le.fit(X_train[_col])
  X_train[_col] = le.transform(X_train[_col])
  X_test[_col] = le.transform(X_test[_col])

### (4) 데이터 모형 구축

#### 434-01 : 훈련데이터 분할

In [ ]:
##### 훈련데이터 분할
# y의 클래스 비율에 맞게 분할 stratify

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)

#### 434-02 : 수치형 변수 - 데이터 스케일링

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_tr[COL_NUM] = scaler.fit_transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

#### 434-03 : 랜덤포레스트 학습

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_tr, y_tr.values.ravel())

RandomForestClassifier()

#### 434-04 : XGBoost 학습 1

In [ ]:
from xgboost import XGBClassifier

model_xgb1 = XGBClassifier()
model_xgb1.fit(X_tr, y_tr.values.ravel())

XGBClassifier()

#### 434-05 : XGBoost 학습 2

In [ ]:
model_xgb2 = XGBClassifier(n_estimators=1000, learning_rate=0.1, max_depth=10)
model_xgb2.fit(X_tr, y_tr.values.ravel(), early_stopping_rounds=50, eval_metric='auc', eval_set=[(X_val, y_val)], verbose=10)

[0]	validation_0-auc:0.905394
Will train until validation_0-auc hasn't improved in 50 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[10]	validation_0-auc:0.911091
[20]	validation_0-auc:0.915016
[30]	validation_0-auc:0.918917
[40]	validation_0-auc:0.920678
[50]	validation_0-auc:0.921773
[60]	validation_0-auc:0.923079
[70]	validation_0-auc:0.923802
[80]	validation_0-auc:0.92431
[90]	validation_0-auc:0.924592
[100]	validation_0-auc:0.924642
[110]	validation_0-auc:0.924601
[120]	validation_0-auc:0.924593
[130]	validation_0-auc:0.924569
[140]	validation_0-auc:0.924022
Stopping. Best iteration:
[97]	validation_0-auc:0.924886



XGBClassifier(max_depth=10, n_estimators=1000)

#### 434-06 : 모델 성능 확인

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_rf = model_rf.predict_proba(X_val)
y_pred_xgb1 = model_xgb1.predict_proba(X_val)

score_rf = roc_auc_score(y_val, y_pred_rf[:, 1])
score_xgb1 = roc_auc_score(y_val, y_pred_xgb1[:, 1]) 

print(score_rf)
print(score_xgb1)

0.8909197415221585
0.9182365417717722


#### 434-07 : 변수 중요도 확인

In [ ]:
pd.DataFrame({'feature': X_tr.columns, 'fi_rf': model_rf.feature_importances_, 'fi_xgb': model_xgb1.feature_importances_})

,feature,fi_rf,fi_xgb
0,age,0.221153,0.066392
1,workclass,0.051042,0.015482
2,education_num,0.138100,0.145672
3,marital_status,0.064809,0.148654
4,occupation,0.089302,0.045329
5,relationship,0.122120,0.333530
6,race,0.017113,0.010633
7,sex,0.011459,0.032497
8,capital_gain,0.100674,0.108042
9,capital_loss,0.028455,0.040944


#### 434-08 : 변수 제거

In [ ]:
COL_DEL = ['capital_gain_yn', 'capital_loss_yn']

X_tr = X_tr.drop(COL_DEL, axis=1)
X_val = X_val.drop(COL_DEL, axis=1)
X_test = X_test.drop(COL_DEL, axis=1)

#### 434-09 : 랜덤 포레스트 하이퍼파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_params = { 
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_cv = GridSearchCV(estimator=model_rf, param_grid=grid_params, cv=5)
rf_cv.fit(X_train, y_train.values.ravel()) 

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

#### 434-10 : CV 결과 확인 1

In [ ]:
print(pd.DataFrame(rf_cv.cv_results_).head())

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.341199      0.007376         0.029468        0.001474   
1       0.680139      0.012316         0.057113        0.001736   
2       1.339999      0.014353         0.105718        0.002818   
3       0.337498      0.004352         0.030323        0.001950   
4       0.675741      0.012524         0.055056        0.001428   

  param_max_depth param_min_samples_leaf param_min_samples_split  \
0               5                      1                       2   
1               5                      1                       2   
2               5                      1                       2   
3               5                      1                       5   
4               5                      1                       5   

  param_n_estimators                                             params  \
0                 50  {'max_depth': 5, 'min_samples_leaf': 1, 'min_s...   
1                100  {'max_depth': 5,

#### 434-11 : CV 결과 확인 2

In [ ]:
rf_cv.best_params_

{'max_depth': 15,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 50}

#### 434-12 : 랜덤포레스트 학습 및 결과 확인 

In [ ]:
model_rf2 = RandomForestClassifier(n_estimators=50
                                   , max_depth=15
                                   , min_samples_leaf=1
                                   , min_samples_split=5)
model_rf2.fit(X_tr, y_tr.values.ravel())

y_pred_rf2 = model_rf2.predict_proba(X_val)
score_rf2 = roc_auc_score(y_val, y_pred_rf2[:, 1])
print(score_rf2)

0.9152708749105954


#### 434-13 : XGBoost 하이퍼파라미터 튜닝

In [ ]:
grid_params = {'max_depth': [3, 5, 7, 10], 
               'min_child_weight': [1, 2], 
               'colsample_bytree': [0.6, 0.8],
               'subsample': [0.6, 0.8]}

xgb_cv = GridSearchCV(estimator=model_xgb1, param_grid=grid_params, cv=5)
xgb_cv.fit(X_tr, y_tr.values.ravel())

GridSearchCV(cv=5, estimator=XGBClassifier(),
             param_grid={'colsample_bytree': [0.6, 0.8],
                         'max_depth': [3, 5, 7, 10], 'min_child_weight': [1, 2],
                         'subsample': [0.6, 0.8]})

#### 434-14 : CV 결과 확인 3

In [ ]:
xgb_cv.best_params_

{'colsample_bytree': 0.6,
 'max_depth': 7,
 'min_child_weight': 1,
 'subsample': 0.8}

#### 434-15 : XGBoost 학습 및 결과 확인

In [ ]:
params = {'colsample_bytree': 0.6,
          'max_depth': 7,
          'min_child_weight': 1,
          'subsample': 0.8}

model_xgb3 = XGBClassifier(n_estimators=1000, learning_rate=0.05)
model_xgb3.set_params(**params)

model_xgb3.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='auc', eval_set=[(X_val, y_val)], verbose=10)

print(model_xgb3.best_score)

[0]	validation_0-auc:0.865221
Will train until validation_0-auc hasn't improved in 50 rounds.


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[10]	validation_0-auc:0.910082
[20]	validation_0-auc:0.913779
[30]	validation_0-auc:0.915745
[40]	validation_0-auc:0.917288
[50]	validation_0-auc:0.918281
[60]	validation_0-auc:0.919243
[70]	validation_0-auc:0.920479
[80]	validation_0-auc:0.921141
[90]	validation_0-auc:0.922188
[100]	validation_0-auc:0.922643
[110]	validation_0-auc:0.923314
[120]	validation_0-auc:0.923896
[130]	validation_0-auc:0.92446
[140]	validation_0-auc:0.924846
[150]	validation_0-auc:0.925091
[160]	validation_0-auc:0.925308
[170]	validation_0-auc:0.925587
[180]	validation_0-auc:0.925721
[190]	validation_0-auc:0.92588
[200]	validation_0-auc:0.925881
[210]	validation_0-auc:0.925939
[220]	validation_0-auc:0.925968
[230]	validation_0-auc:0.926082
[240]	validation_0-auc:0.926158
[250]	validation_0-auc:0.926232
[260]	validation_0-auc:0.92607
[270]	validation_0-auc:0.926089
[280]	validation_0-auc:0.925998
[290]	validation_0-auc:0.925813
[300]	validation_0-auc:0.925793
Stopping. Best iteration:
[251]	validation_0-auc:0.9

#### 434-16 : 결과 제출

In [ ]:
pred = model_xgb3.predict_proba(X_test)[:,1]
pd.DataFrame({'index': X_test.index, 'target': pred}).to_csv('003000000.csv', index=False)